# Schema linking

In [ ]:
import os
import json
import torch
import transformers
import argparse
import torch.optim as optim

from tqdm import tqdm
from copy import deepcopy
from tokenizers import AddedToken
from utils.classifier_metric.evaluator import cls_metric, auc_metric
from torch.utils.data import DataLoader
from transformers import RobertaTokenizerFast, XLMRobertaTokenizerFast
from utils.classifier_model import MyClassifier
from utils.classifier_loss import ClassifierLoss
from transformers.trainer_utils import set_seed
from torch.utils.tensorboard import SummaryWriter
from utils.load_dataset import ColumnAndTableClassifierDataset

In [ ]:
text2natsql_model_save_path="./models/text2natsql-t5-base/checkpoint-14352"
text2natsql_model_bs=16

In [1]:
from infection.databases import SQL3Database
import sqlite3

def connect_func(database_name: str, database_type: str = 'sqlite3'):
    try:
        connection = SQL3Database(database_name)
        return connection
    except sqlite3.Error as e:
        print(f"Error connecting to the database: {e}")
        return None


In [3]:
question = "What's the statistic code used for fully vaccinated?"

In [4]:
connection = connect_func('../data/example-data/example-covid-vaccinations.sqlite3')
schemas = connection.format_schemas(add_examples=1)
# query = sprompt.SQL_QUERY_PROMPT_TEMPLATE.format(question=question, db_schema=schemas, tables_hints=None)
prompt = """
{schemas}
**Using valid SQLite, answer the following questions for the tables provided above**.
-- {question}
```sql
"""
query = prompt.format(schemas=schemas, question=question)
print(query)


CREATE TABLE covid_vaccinations (
 	STATISTIC_CODE varchar(10),
	Statistic_Label varchar(30),
	TLIST(M1) INT,
	Month varchar(20),
	C03898V04649 varchar(30),
	Local Electoral Area varchar(50),
	C02076V03371 varchar(10),
	Age Group varchar(30),
	UNIT varchar(10),
	VALUE float,
);
SELECT * FROM covid_vaccinations LIMIT 1;
| STATISTIC_CODE   | Statistic_Label   |   TLIST(M1) | Month        | C03898V04649                         | Local Electoral Area                 |   C02076V03371 | Age Group    | UNIT   |   VALUE |
|------------------|-------------------|-------------|--------------|--------------------------------------|--------------------------------------|----------------|--------------|--------|---------|
| CDC45C01         | Fully Vaccinated  |      202101 | 2021 January | 2ae19629-3eff-13a3-e055-000000000001 | Borris-In-Ossory-Mountmellick, Laois |            247 | 5 - 11 years | %      |       0 |


**Using valid SQLite, answer the following questions for the tables provided ab